In [ ]:
# Astropy tools
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord

# Moc and HEALPix tools
import cdshealpix
import mocpy
from mocpy import MOC, WCS

# For plots
import matplotlib.pyplot as plt

import numpy as np

print("healpix version : ", cdshealpix.__version__)
print("mocpy version : ", mocpy.__version__)

In [ ]:
ipix = np.arange(12 * 4**3, dtype=np.uint64)
depth = 3
delta_depth = 2

In [ ]:
help(cdshealpix.external_neighbours)
edges, corners = cdshealpix.external_neighbours(ipix, depth, delta_depth)

In [ ]:
ipix_corner_cells = corners[corners >= 0].ravel().astype(int)
ipix_border_cells = edges.ravel().astype(int)

In [ ]:
help(MOC.from_healpix_cells)

In [ ]:
# Create the moc from corner cells
depth_corner_cells = np.ones(ipix_corner_cells.shape, dtype=np.uint8) * (
    depth + delta_depth
)
moc_from_corner_cells = MOC.from_healpix_cells(ipix=ipix_corner_cells, depth=depth_corner_cells, max_depth=depth + delta_depth)

In [ ]:
# Create the moc from border cells
depth_border_cells = np.ones(ipix_border_cells.shape, dtype=np.uint8) * (
    depth + delta_depth
)
moc_from_border_cells = MOC.from_healpix_cells(ipix=ipix_border_cells, depth=depth_border_cells, max_depth=depth + delta_depth)

In [ ]:
# Plot the MOC using matplotlib
fig = plt.figure(111, figsize=(10, 10))
# Define a astropy WCS from the mocpy.WCS class
with WCS(
    fig,
    fov=120 * u.deg,
    center=SkyCoord(0, 0, unit="deg", frame="icrs"),
    coordsys="icrs",
    rotation=Angle(0, u.degree),
    projection="SIN",
) as wcs:
    ax = fig.add_subplot(1, 1, 1, projection=wcs)

    moc_from_corner_cells.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="green")
    moc_from_border_cells.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="red")

plt.xlabel("ra")
plt.ylabel("dec")
plt.title("neighbours")
plt.grid(color="black", linestyle="dotted")
plt.show()